# 5. グラフィカルモデル（問題62～75）

In [51]:
import copy
import japanize_matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy
from matplotlib.pyplot import imshow
from numpy.random import randn
from scipy import stats
from scipy import sparse
from sklearn.covariance import graphical_lasso
from IPython.core.display import display, SVG
import igraph
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression
import scipy.stats as ss

In [11]:
def inner_prod(x, y):
    return np.dot(x, y)

In [12]:
def soft_th(lambd, x):
    return np.sign(x) * np.maximum(np.abs(x) - lambd, 0)

In [25]:
def fused_dual(y, D):
    m = D.shape[0]
    lambda_seq = np.zeros(m)
    s = np.zeros(m)
    alpha = np.zeros((m, m))
    alpha[0, :] = np.linalg.pinv(D @ D.T) @ D @ y
    for j in range(m):
        if np.abs(alpha[0, j]) > lambda_seq[0]:
            lambda_seq[0] = np.abs(alpha[0, j])
            index = [j]
            if alpha[0, j] > 0:
                s[j] = 1
            else:
                s[j] = -1
    f_s = list(range(m))
    for k in range(1, m):
        sub_s = list(set(f_s) - set(index))
        U = np.linalg.pinv(D[sub_s, :] @ D[sub_s, :].T)
        V = D[sub_s, :] @ D[index, :].T
        u = U @ D[sub_s, :] @ y
        v = U @ V @ s[index]
        t = u / (v+1)
        for i in range(0, m-k):
            if t[i] > lambda_seq[k]:
                lambda_seq[k] = t[i]
                h = i
                r = 1
        t = u / (v-1)
        for i in range(0, m-k):
            if t[i] > lambda_seq[k]:
                lambda_seq[k] = t[i]
                h = i
                r = -1
        alpha[k, index] = lambda_seq[k] * s[index]
        alpha[k, sub_s] = u - lambda_seq[k] * v
        h = sub_s[h]
        index.append(h)
        if r == 1:
            s[h] = 1
        else:
            s[h] = -1
    return [alpha, lambda_seq]

In [26]:
def fused_prime(y, D):
    alpha, lambda_seq = fused_dual(y, D)
    m = D.shape[0]
    return [np.tile(y, (m, 1)).T - D.T @ alpha.T, lambda_seq]

In [27]:
# 第4章のfused_dual, fused_primeを用いる
def fused_2(y, D, lam_0):
    beta, lam = fused_prime(y, D)
    m, p = D.shape
    i = 0
    for k in range(1, m):
        if lam[k-1] < lam_0 <= lam[k]:
            i = k
    if lam_0 > lam[m-1]:
        beta_0 = beta[m-1, ]
    elif i == 0:
        beta_0 = beta[0, ]
    else:
        beta_0 = (beta[i-1, ]
                  + (lam_0 - lam[i-1]) / (lam[i] - lam[i-1]) * (beta[i, ] - beta[i-1, ]))
    return beta_0

In [28]:
# 大きさが3以上でないとfused_2は稼働しない
def b_fused(y, lambd):
    if y[0] > y[1] + 2 * lambd:
        a = y[0] - lambd; b = y[1] + lambd
    elif y[0] < y[1] - 2 * lambd:
        a = y[0] + lambd; b = y[1] - lambd
    else:
        a = (y[0] + y[1]) / 2; b = a
    return [a, b]

### 70

In [13]:
def graph_lasso(s, lambd=0):
    s = np.array(s)
    W = s; p = s.shape[1]; beta = np.zeros((p-1, p)); w = s.shape[0]
    beta_out = beta; eps_out = 1
    while eps_out > 0.01:
        for j in range(p):
            a = np.delete(np.delete(W, j, 0), j, 1); b = np.delete(s, j, 0)[:, j]
            beta_in = beta[:, j]; eps_in = 1
            while eps_in > 0.01:
                for h in range(p - 1):
                    cc = b[h] - inner_prod(np.delete(a, h, 1)[h, :],
                                           np.delete(beta, h, 0)[:, j])
                    beta[h, j] = soft_th(lambd, cc) / a[h, h]
                eps_in = np.max(beta[:, j] - beta_in); beta_in = beta[:, j]
            m = list(np.arange(j)); n = list(np.arange(j+1, w)); z = m + n
            W[z, j] = np.dot(a, beta[:, j])
        eps_out = np.max(beta - beta_out); beta_out = beta
    theta = np.zeros((p, p))
    for j in range(p - 1):
        m1 = list(np.arange(j)); n1 = list(np.arange(j+1, p)); z1 = m1 + n1
        theta[j, j] = 1 / (W[j, j] - np.dot(np.delete(W, j, 1)[j, :], beta[:, j]))
        theta[z1, j] = -beta[:, j] * theta[j, j]
    return theta

In [14]:
Theta = np.array([   2,  0.6,    0,    0,  0.5,  0.6,    2, -0.4,  0.3,    0,
                     0, -0.4,    2, -0.2,    0,    0,  0.3, -0.2,    2, -0.2,
                   0.5,    0,    0, -0.2,    2]).reshape(-1, 5)
Sigma = np.linalg.inv(Theta)
meanvec = np.repeat(0, 5)
dat = np.random.multivariate_normal(meanvec, Sigma, 20)

In [15]:
# サンプル行列を生成
s = np.dot(dat.T, dat) / dat.shape[0]

In [16]:
print(Theta)
print(graph_lasso(s))
print(graph_lasso(s, lambd=0.01))

[[ 2.   0.6  0.   0.   0.5]
 [ 0.6  2.  -0.4  0.3  0. ]
 [ 0.  -0.4  2.  -0.2  0. ]
 [ 0.   0.3 -0.2  2.  -0.2]
 [ 0.5  0.   0.  -0.2  2. ]]
[[ 3.2944715   0.19418198 -0.07692045  0.40723872  0.        ]
 [ 0.2083362   1.52661781 -0.46249813  0.00421346  0.        ]
 [-0.14719216 -0.44146561  2.00228424 -0.22876887  0.        ]
 [ 0.50885491  0.03597965 -0.27987639  1.4188403   0.        ]
 [ 1.00905069 -0.37691633 -0.40063534  0.03073113  0.        ]]
[[ 3.1959854   0.15270883 -0.02636389  0.36742554  0.        ]
 [ 0.16242552  1.49387774 -0.42096484 -0.          0.        ]
 [-0.0776389  -0.40945508  1.96131445 -0.2013278   0.        ]
 [ 0.44380262  0.00328657 -0.23599099  1.40213153  0.        ]
 [ 0.9313522  -0.36332305 -0.36828279  0.          0.        ]]


### 71

In [43]:
import pandas as pd

In [44]:
def adj(mat):
    p = mat.shape[1]; ad = np.zeros((p, p))
    for i in range(p - 1):
        for j in range((i + 1), p):
            if mat[i, j] == 0:
                ad[i, j] = 0
            else:
                ad[i, j] = 1
    g = igraph.Graph.Adjacency(ad.tolist(), mode=igraph.ADJ_MAX)
    g.vs["label"] = list(range(g.vcount()))
    return igraph.plot(g, bbox=(300, 300))

In [45]:
df = pd.read_csv("breastcancer.csv")
df.drop(df.columns[len(df.columns) - 1], axis=1, inplace=True)
df = df.values

In [52]:
w = np.zeros((250, 1000))
for i in range(1000):
    w[:, i] = df[:, i]
x = w; s = np.dot(x.T, x) / 250
fit = graphical_lasso(s, 0.75)
print(np.sum(list(map(lambda x: x == 0, fit[1]))))
y = pd.DataFrame(columns=["y"]); z = pd.DataFrame(columns=["z"])
for i in range(999):
    for j in range((i + 1), 1000):
        if fit[1][i, j] != 0:
            y = y.append(pd.DataFrame({"y": [i]}))
            z = z.append(pd.DataFrame({"z": [j]}))
y.index = np.arange(1, len(y) + 1)
z.index = np.arange(1, len(z) + 1)
edges = pd.concat([y, z], axis=1)
edges.to_csv("edges.csv")

KeyboardInterrupt: 

### 72

In [47]:
df = pd.read_csv("breastcancer.csv")
df.drop(df.columns[len(df.columns) - 1], axis=1, inplace=True)
df = df.values

In [48]:
n = 250; p = 50
w = np.zeros((n, p))
for i in range(p):
    w[:, i] = df[:, i]
x = w[:, range(p)]; lambd = 0.1
model = list()
for j in range(p):
    m2 = list(np.arange(j)); n2 = list(np.arange(j+1, p)); z2 = m2 + n2
    model.append(ElasticNet(alpha=lambd, l1_ratio=1).fit(X=x[:, z2], y=x[:, j]))
ad = np.zeros((p, p))
for i in range(p):
    for j in range(p - 1):
        k = j
        if j >= i:
            k = j + 1
        if model[i].coef_[j] != 0:
            ad[i, k] = 1
        else:
            ad[i, k] = 0

In [57]:
# ANDの場合
for i in range(p - 1):
    for j in range(i + 1, p):
        if ad[i, j] != ad[i, j]:
            ad[i, j] = 0; ad[j, i] = 0
u = list(); v = list()
for i in range(p - 1):
    for j in range(i + 1, p):
        if ad[i, j] == 1:
            u.append(i)
            v.append(j)
print(u, v)
print(ad)
adj(ad)

[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 8, 8, 8, 8, 8, 9, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 13, 14, 15, 15, 15, 15, 15, 17, 17, 17, 17, 18, 18, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 25, 26, 26, 26, 26, 27, 27, 27, 28, 28, 28, 28, 29, 29, 29, 30, 30, 31, 31, 32, 33, 34, 34, 36, 37, 37, 37, 37, 39, 41, 43, 44, 46] [11, 17, 23, 34, 39, 40, 4, 11, 22, 25, 26, 33, 34, 38, 40, 8, 18, 7, 16, 30, 17, 20, 30, 31, 14, 21, 23, 25, 30, 29, 30, 43, 47, 11, 16, 13, 18, 20, 23, 43, 17, 41, 17, 23, 31, 40, 48, 21, 30, 40, 44, 14, 30, 17, 26, 31, 40, 48, 23, 26, 34, 40, 19, 43, 43, 47, 23, 30, 37, 26, 34, 35, 31, 34, 34, 40, 44, 47, 38, 39, 49, 36, 39, 41, 48, 30, 31, 39, 29, 35, 44, 49, 30, 44, 46, 33, 38, 32, 48, 48, 35, 39, 40, 41, 38, 44, 45, 49, 40, 48, 47, 45, 47]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 

AttributeError: Plotting not available; please install pycairo or cairocffi

In [56]:
w = np.zeros((250, 1000))
for i in range(1000):
    w[:, i] = df[:, i]
w = (np.sign(w) + 1) / 2
p = 50; x = w[:, range(p)]; lambd = 0.15
x[x == 0] = -1
model = list()
for j in range(p):
    m3 = list(np.arange(j)); n3 = list(np.arange(j+1, p)); z3 = m3 + n3
    model.append(LogisticRegression(
        C=(1 / (250*lambd)), penalty="l1", solver="liblinear",
        fit_intercept=True).fit(X=x[:, z3], y=x[:, j]))
print(model[1].coef_)
ad = np.zeros((p, p))
for i in range(p):
    for j in range(p - 1):
        k = j
        if j >= i:
            k = j + 1
        if model[i].coef_[:, j] != 0:
            ad[i, k] = 1
        else:
            ad[i, k] = 0
for i in range(p - 1):
    for j in range(i + 1, p):
        if ad[i, j] != ad[i, j]:
            ad[i, j] = 0; ad[j, i] = 0
print(ad)
print(np.sum(ad)); adj(ad)

[[0.         0.         0.         0.03116839 0.         0.
  0.         0.         0.         0.         0.03749406 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.03077982 0.         0.
  0.05842514 0.00050559 0.         0.         0.         0.
  0.         0.         0.14072457 0.13816426 0.         0.
  0.         0.06911837 0.         0.1585439  0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
210.0


AttributeError: Plotting not available; please install pycairo or cairocffi

### 74

In [58]:
def graph_fused(y=[], lambd1=None, lambd2=None):
    K = len(y)
    if K == 1: theta = y
    elif K == 2: theta = b_fused(y, lambd2)
    else:
        y = np.array(y)
        L = K * (K - 1) / 2; D = np.zeros((int(L), K))
        k = 0
        for i in range(K - 1):
            for j in range(i + 1, K):
                D[k, i] = 1; D[k, j] = -1; k = k + 1
        theta = fused_2(y, D, lambd2)
    theta = soft_th(lambd1, theta)
    return theta

In [60]:
# Joint Graphical Lasso
def jgl(X, lambd1, lambd2):
    K = len(X); p = np.array(X[1]).shape[1]; n = np.zeros(K); S = list()
    for k in range(K):
        n[k] = X[k].shape[0]; S.append(np.dot(X[k].T, X[k]) / n[k])
    rho = 1; lambd1 = lambd1 / rho; lambd2 = lambd2 / rho
    Theta = [0] * K
    for k in range(K): Theta[k] = np.diag([1] * p)
    Theta_old = [0] * K
    for k in range(K): Theta_old[k] = np.diag(np.random.normal(size=p))
    U = [0] * K
    for k in range(K): U[k] = np.zeros((p, p))
    Z = [0] * K
    for k in range(K): Z[k] = np.zeros((p, p))
    epsilon = 0; epsilon_old = 1; h = 0
    while np.abs(epsilon - epsilon_old) > 0.0001 * epsilon_old:
        h = h + 1
        Theta_old = c.deepcopy(Theta); epsilon_old = epsilon
        # (a)に関する更新
        for k in range(K):
            mat = S[k] - rho * Z[k] / n[k] + rho * U[k] / n[k]
            u, s, v = np.linalg.svd(mat)
            DD = n[k] / (2 * rho) * (-s + np.sqrt(np.square(s) + 4 * rho / n[k]))
            Theta[k] = np.dot(np.dot(v.T, np.diag(DD)), v)
        # (b)に関する更新
        for i in range(p):
            for j in range(p):
                A = list()
                for k in range(K): A.append(Theta[k][i, j] + U[k][i, j])
                if i == j: B = graph_fused(A, 0, lambd2)
                else: B = graph_fused(A, lambd1, lambd2)
                for k in range(K): Z[k][i, j] = B[k]
        # (c)に関する更新
        for k in range(K):
            U[k] = U[k] + Theta[k] - Z[k]
        # 収束したかどうかの検査
        epsilon = 0
        for k in range(K):
            epsilon_new = np.max(np.abs(Theta[k] - Theta_old[k]))
            if epsilon_new > epsilon: epsilon = epsilon_new
    print("epsilon:", epsilon)
    print("M:", np.abs(epsilon - epsilon_old))
    print("epsilon_old * 0.0001:", epsilon_old * 0.0001)
    print("The number of while loop:", h)
    return Z

### 75

In [65]:
# Joint Graphical Lasso
def jgl(X, lambd1, lambd2):
    K = len(X); p = np.array(X[1]).shape[1]; n = np.zeros(K); S = list()
    for k in range(K):
        n[k] = X[k].shape[0]; S.append(np.dot(X[k].T, X[k]) / n[k])
    rho = 1; lambd1 = lambd1 / rho; lambd2 = lambd2 / rho
    Theta = [0] * K
    for k in range(K): Theta[k] = np.diag([1] * p)
    Theta_old = [0] * K
    for k in range(K): Theta_old[k] = np.diag(np.random.normal(size=p))
    U = [0] * K
    for k in range(K): U[k] = np.zeros((p, p))
    Z = [0] * K
    for k in range(K): Z[k] = np.zeros((p, p))
    epsilon = 0; epsilon_old = 1; h = 0
    while np.abs(epsilon - epsilon_old) > 0.0001 * epsilon_old:
        h = h + 1
        Theta_old = c.deepcopy(Theta); epsilon_old = epsilon
        # (a)に関する更新
        for k in range(K):
            mat = S[k] - rho * Z[k] / n[k] + rho * U[k] / n[k]
            u, s, v = np.linalg.svd(mat)
            DD = n[k] / (2 * rho) * (-s + np.sqrt(np.square(s) + 4 * rho / n[k]))
            Theta[k] = np.dot(np.dot(v.T, np.diag(DD)), v)
        # (b)に関する更新
        for i in range(p):
            for j in range(p):
                A = list()
                for k in range(K): A.append(Theta[k][i, j] + U[k][i, j])
                if i == j:
                    B = A
                else:
                    B = soft_th(lambd1 / rho, A) * np.max(
                        1 - lambd2 / rho / np.sqrt(
                            np.linalg.norm(soft_th(lambd1 / rho, A), 2) ** 2), 0)
                for k in range(K):
                    Z[k][i, j] = B[k]
        # (c)に関する更新
        for k in range(K):
            U[k] = U[k] + Theta[k] - Z[k]
        # 収束したかどうかの検査
        epsilon = 0
        for k in range(K):
            epsilon_new = np.max(np.abs(Theta[k] - Theta_old[k]))
            if epsilon_new > epsilon: epsilon = epsilon_new
    print("epsilon:", epsilon)
    print("M:", np.abs(epsilon - epsilon_old))
    print("epsilon_old * 0.0001:", epsilon_old * 0.0001)
    print("The number of while loop:", h)
    return Z